In [14]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import json
import os
import pandas as pd

In [12]:
PATH_DATA_FOLDER = os.path.join("/home", "ubuntu", "lopezrbn", "goatbots_price_checker", "0_data")
PATH_OTHER_DATA = os.path.join(PATH_DATA_FOLDER, "3_other_data")
PATH_SELL_ALERTS = os.path.join(PATH_OTHER_DATA, "sell_alerts.json")
PATH_BUY_ALERTS = os.path.join(PATH_OTHER_DATA, "buy_alerts.json")

In [9]:
with open("credentials.json") as f:
    credentials = json.load(f)
    sender_email = credentials["sender_email"]
    receiver_email = credentials["receiver_email"]
    password = credentials["password"]

In [29]:
with open(PATH_SELL_ALERTS) as f:
    # sell_alerts = json.load(f)
    sell_alerts = pd.read_json(f, orient="index")
with open(PATH_BUY_ALERTS) as f:
    buy_alerts = pd.read_json(f, orient="index")

In [30]:
sell_alerts['today_vs_range'] = sell_alerts['today_vs_range'].map("{:.0%}".format)
sell_alerts

,name,cardset,foil,range_time,today_vs_range,price_min_range,price_today,price_max_range
107243,Wrath of God,DMR,0,3m,100%,2.23,2.82,2.82


In [31]:
buy_alerts['today_vs_range'] = buy_alerts['today_vs_range'].map("{:.0%}".format)
buy_alerts

,name,cardset,foil,range_time,today_vs_range,price_min_range,price_today,price_max_range
10043,Ancient Tomb,TE,0,3m,19%,18.74,21.49,33.49
18245,Flooded Strand,ONS,0,3m,11%,7.19,8.44,18.99


In [32]:
sell_alerts_html = sell_alerts.to_html(index=False)
sell_alerts_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>name</th>\n      <th>cardset</th>\n      <th>foil</th>\n      <th>range_time</th>\n      <th>today_vs_range</th>\n      <th>price_min_range</th>\n      <th>price_today</th>\n      <th>price_max_range</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>Wrath of God</td>\n      <td>DMR</td>\n      <td>0</td>\n      <td>3m</td>\n      <td>100%</td>\n      <td>2.23</td>\n      <td>2.82</td>\n      <td>2.82</td>\n    </tr>\n  </tbody>\n</table>'

In [33]:
buy_alerts_html = buy_alerts.to_html(index=False)
buy_alerts_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>name</th>\n      <th>cardset</th>\n      <th>foil</th>\n      <th>range_time</th>\n      <th>today_vs_range</th>\n      <th>price_min_range</th>\n      <th>price_today</th>\n      <th>price_max_range</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>Ancient Tomb</td>\n      <td>TE</td>\n      <td>0</td>\n      <td>3m</td>\n      <td>19%</td>\n      <td>18.74</td>\n      <td>21.49</td>\n      <td>33.49</td>\n    </tr>\n    <tr>\n      <td>Flooded Strand</td>\n      <td>ONS</td>\n      <td>0</td>\n      <td>3m</td>\n      <td>11%</td>\n      <td>7.19</td>\n      <td>8.44</td>\n      <td>18.99</td>\n    </tr>\n  </tbody>\n</table>'

In [3]:
# Create message
message = MIMEMultipart("alternative")
message["Subject"] = "Daily Price Update"
message["From"] = sender_email
message["To"] = receiver_email

In [34]:
# Email body
body = f"""
<html>
    <head></head>
    <body>
        <p>Here's the daily update on prices...</p>
        <h2>Sell Alerts</h2>
        {sell_alerts_html}
        <h2>Buy Alerts</h2>
        {buy_alerts_html}
    </body>
</html>
"""
mime_text = MIMEText(body, "html")
message.attach(mime_text)

In [35]:
# Send the email
with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, message.as_string())